## Research Agent

In [44]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper

# Initialize the LLM
llm = ChatOpenAI(temperature=0.1, model_name="gpt-4o-mini")

# Tool to search Wikipedia
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic to search on Wikipedia.")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Use this tool to search for topics on Wikipedia."
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query: str):
        response = requests.get(
            f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json"
        )
        search_results = response.json()
        if "query" in search_results and "search" in search_results["query"]:
            return search_results["query"]["search"][0]["snippet"]
        return "No results found on Wikipedia."


# Tool to search using DuckDuckGo
class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic to search on DuckDuckGo.")


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "Use this tool to search for information using DuckDuckGo."
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query: str):
        ddg = DuckDuckGoSearchAPIWrapper()
        try:
            return ddg.run(query)
        except Exception as e:
            return f"Error searching DuckDuckGo: {str(e)}"

def create_search_agent():
    tools = [
        WikipediaSearchTool(),
        DuckDuckGoSearchTool(),
    ]

    search_agent = initialize_agent(
        llm=llm,
        verbose=True,
        agent=AgentType.OPENAI_FUNCTIONS,
        tools=tools,
    )

    return search_agent

search_agent = create_search_agent()

In [45]:
search_agent.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor'}`


Error searching DuckDuckGo: 
Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


In February 2024, a malicious <span class="searchmatch">backdoor</span> was introduced to the Linux build of the <span class="searchmatch">xz</span> utility within the liblzma library in versions 5.6.0 and 5.6.1 by an
Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor security'}`


In February 2024, a malicious <span class="searchmatch">backdoor</span> was introduced to the Linux build of the <span class="searchmatch">xz</span> utility within the liblzma library in versions 5.6.0 and 5.6.1 by an
Invoking: `DuckDuckGoSearchTool` with `{'query': 'XZ backdoor Linux utility liblzma'}`


Error searching DuckDuckGo: The XZ backdoor refers to a security vulnerability that was introduced in February 2024 in the Linux build of the `xz` utility, which is part of the `liblzma` library. T

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor refers to a security vulnerability that was introduced in February 2024 in the Linux build of the `xz` utility, which is part of the `liblzma` library. This backdoor was found in versions 5.6.0 and 5.6.1 of the library.\n\nThe backdoor allowed unauthorized access to systems using these versions of the `xz` utility, potentially compromising the security of affected systems. The introduction of the backdoor was likely the result of malicious intent, and it raised significant concerns within the security community regarding the integrity of software packages and the importance of verifying the authenticity of software updates.\n\nIf you need more detailed information or specific aspects of the XZ backdoor, please let me know!'}

In [58]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool, DuckDuckGoSearchRun
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType
# from langchain.utilities import DuckDuckGoSearchAPIWrapper

# Initialize the LLM
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o-mini")

# Tool to search Wikipedia
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic to search on Wikipedia.")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Use this tool to search for topics on Wikipedia."
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query: str):
        response = requests.get(
            f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json"
        )
        search_results = response.json()
        if "query" in search_results and "search" in search_results["query"]:
            return search_results["query"]["search"][0]["snippet"]
        return "No results found on Wikipedia."
    
import requests


class ArxivSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic or keywords to search for in arXiv.")
    max_results: int = Field(
        default=5, description="The maximum number of results to fetch from arXiv."
    )


class ArxivSearchTool(BaseTool):
    name = "ArxivSearchTool"
    description = "Search for academic papers in arXiv."
    args_schema: type = ArxivSearchToolArgsSchema

    def _run(self, query: str, max_results: int = 5):
        try:
            # Construct the arXiv API query
            url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}"
            response = requests.get(url)

            if response.status_code == 200:
                # Parse the response
                entries = []
                for entry in response.text.split("<entry>")[1:]:
                    title_start = entry.find("<title>") + len("<title>")
                    title_end = entry.find("</title>")
                    summary_start = entry.find("<summary>") + len("<summary>")
                    summary_end = entry.find("</summary>")
                    link_start = entry.find("<id>") + len("<id>")
                    link_end = entry.find("</id>")

                    title = entry[title_start:title_end].strip()
                    summary = entry[summary_start:summary_end].strip()
                    link = entry[link_start:link_end].strip()

                    entries.append(f"Title: {title}\nSummary: {summary}\nLink: {link}")

                return "\n\n".join(entries)
            else:
                return f"Error: Unable to fetch data from arXiv. Status code: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"


def create_search_agent():
    tools = [
        WikipediaSearchTool(),
        ArxivSearchTool(),
    ]

    search_agent = initialize_agent(
        llm=llm,
        verbose=True,
        agent=AgentType.OPENAI_FUNCTIONS,
        tools=tools,
    )

    return search_agent

search_agent = create_search_agent()

In [56]:
search_agent.invoke("Research about the XZ backdoor")



> Entering new AgentExecutor chain...

Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor'}`


In February 2024, a malicious <span class="searchmatch">backdoor</span> was introduced to the Linux build of the <span class="searchmatch">xz</span> utility within the liblzma library in versions 5.6.0 and 5.6.1 by an
Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor malware'}`


library <span class="searchmatch">xz</span>&quot;. The Register. Archived from the original on 1 April 2024. Retrieved 1 April 2024. Gatlan, Sergiu. &quot;Red Hat warns of <span class="searchmatch">backdoor</span> in <span class="searchmatch">XZ</span> tools used
Invoking: `WikipediaSearchTool` with `{'query': 'XZ backdoor security incident'}`


In February 2024, a malicious <span class="searchmatch">backdoor</span> was introduced to the Linux build of the <span class="searchmatch">xz</span> utility within the liblzma library in versions 5.6.0 and 5.6.1 by an
Invoking: `ArxivSearchTool` with `X

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor refers to a security incident involving a malicious backdoor that was introduced into the Linux build of the XZ utility, which is part of the liblzma library. This backdoor was discovered in February 2024 and affected versions 5.6.0 and 5.6.1 of the library. The backdoor allowed attackers to execute commands remotely on vulnerable servers using SSH without requiring prior authentication.\n\nThis incident is notable as it highlights the risks associated with supply-chain attacks, where malicious code can be inserted into widely-used software packages. Security experts and organizations, such as Red Hat, have issued warnings and discussed mitigation strategies in response to this vulnerability.\n\n### Key Points:\n- **Backdoor Introduction**: The backdoor was integrated into the XZ utility, which is commonly used for data compression in Linux environments.\n- **Impact**: The backdoor provided unauthorized access to s

In [67]:
import os
import requests
from typing import Type
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from pydantic import BaseModel, Field
from langchain.agents import initialize_agent, AgentType

# Initialize the LLM
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4o")

# Tool to search Wikipedia
class WikipediaSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic to search on Wikipedia.")


class WikipediaSearchTool(BaseTool):
    name = "WikipediaSearchTool"
    description = "Use this tool to search for topics on Wikipedia."
    args_schema: Type[WikipediaSearchToolArgsSchema] = WikipediaSearchToolArgsSchema

    def _run(self, query: str):
        response = requests.get(
            f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={query}&format=json"
        )
        search_results = response.json()
        if "query" in search_results and "search" in search_results["query"]:
            return search_results["query"]["search"][0]["snippet"]
        return "No results found on Wikipedia."
    
import requests


class ArxivSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The topic or keywords to search for in arXiv.")
    max_results: int = Field(
        default=5, description="The maximum number of results to fetch from arXiv."
    )


class ArxivSearchTool(BaseTool):
    name = "ArxivSearchTool"
    description = "Search for academic papers in arXiv."
    args_schema: type = ArxivSearchToolArgsSchema

    def _run(self, query: str, max_results: int = 5):
        try:
            # Construct the arXiv API query
            url = f"http://export.arxiv.org/api/query?search_query=all:{query}&start=0&max_results={max_results}"
            response = requests.get(url)

            if response.status_code == 200:
                # Parse the response
                entries = []
                for entry in response.text.split("<entry>")[1:]:
                    title_start = entry.find("<title>") + len("<title>")
                    title_end = entry.find("</title>")
                    summary_start = entry.find("<summary>") + len("<summary>")
                    summary_end = entry.find("</summary>")
                    link_start = entry.find("<id>") + len("<id>")
                    link_end = entry.find("</id>")

                    title = entry[title_start:title_end].strip()
                    summary = entry[summary_start:summary_end].strip()
                    link = entry[link_start:link_end].strip()

                    entries.append(f"Title: {title}\nSummary: {summary}\nLink: {link}")

                return "\n\n".join(entries)
            else:
                return f"Error: Unable to fetch data from arXiv. Status code: {response.status_code}"
        except Exception as e:
            return f"Error: {str(e)}"


class FileSaveToolArgsSchema(BaseModel):
    content: str = Field(description="The content to save to a file.")
    file_name: str = Field(
        default=None,
        description="The name of the file to save. If not provided, a default file name 'default_output.txt' will be used.",
    )


class FileSaveTool(BaseTool):
    name = "FileSaveTool"
    description = "Save content to a file."
    args_schema: type = FileSaveToolArgsSchema

    def _run(self, content: str, file_name: str = None):
        if not file_name:
            file_name = "default_output.txt"

        if not file_name.endswith(".txt"):
            return "Error: The file name must end with .txt."

        try:
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(content)
            return f"Content successfully saved to {file_name}"
        except Exception as e:
            return f"Error saving content to file: {str(e)}"


def create_search_agent():
    tools = [
        WikipediaSearchTool(),
        ArxivSearchTool(),
        FileSaveTool(),
    ]

    search_agent = initialize_agent(
        llm=llm,
        verbose=True,
        agent=AgentType.OPENAI_FUNCTIONS,
        tools=tools,
    )

    return search_agent

search_agent = create_search_agent()

In [65]:
file_tool = FileSaveTool()
result = file_tool._run(content="Sample text", file_name="output.txt")
print(result)

Content successfully saved to output.txt


In [66]:
search_agent.invoke("Research about the XZ backdoor and save the results to a file 'XZ_backdoor_research.txt'")



> Entering new AgentExecutor chain...

Invoking: `ArxivSearchTool` with `XZ backdoor`


Title: On the critical path to implant backdoors and the effectiveness of
  potential mitigation techniques: Early learnings from XZ
Summary: An emerging supply-chain attack due to a backdoor in XZ Utils has been
identified. The backdoor allows an attacker to run commands remotely on
vulnerable servers utilizing SSH without prior authentication. We have started
to collect available information with regards to this attack to discuss current
mitigation strategies for such kinds of supply-chain attacks. This paper
introduces the critical attack path of the XZ backdoor and provides an overview
about potential mitigation techniques related to relevant stages of the attack
path.
Link: http://arxiv.org/abs/2404.08987v1

Title: Weaponizing Disinformation Against Critical Infrastructures
Summary: For nearly a decade, disinformation has dominated social debates, with its
harmful impacts growing more evident

{'input': "Research about the XZ backdoor and save the results to a file 'XZ_backdoor_research.txt'",
 'output': "The research about the XZ backdoor has been completed and saved to the file 'XZ_backdoor_research.txt'."}

In [68]:
search_agent.invoke("Research about the ReAct Agent and save the results to a file 'ReAct Agent_research.txt'")



> Entering new AgentExecutor chain...

Invoking: `ArxivSearchTool` with `ReAct Agent`


Title: On the Brittle Foundations of ReAct Prompting for Agentic Large Language
  Models
Summary: The reasoning abilities of Large Language Models (LLMs) remain a topic of
debate. Some methods such as ReAct-based prompting, have gained popularity for
claiming to enhance sequential decision-making abilities of agentic LLMs.
However, it is unclear what is the source of improvement in LLM reasoning with
ReAct based prompting. In this paper we examine these claims of ReAct based
prompting in improving agentic LLMs for sequential decision-making. By
introducing systematic variations to the input prompt we perform a sensitivity
analysis along the claims of ReAct and find that the performance is minimally
influenced by the "interleaving reasoning trace with action execution" or the
content of the generated reasoning traces in ReAct, contrary to original claims
and common usage. Instead, the performance o

{'input': "Research about the ReAct Agent and save the results to a file 'ReAct Agent_research.txt'",
 'output': 'The research on the ReAct Agent has been conducted and saved to the file \'ReAct Agent_research.txt\'. Here are some of the key findings from the research:\n\n1. **On the Brittle Foundations of ReAct Prompting for Agentic Large Language Models**: This paper examines the claims of ReAct-based prompting in improving agentic LLMs for sequential decision-making and finds that the performance is minimally influenced by the interleaving reasoning trace with action execution. Instead, it is driven by the similarity between input example tasks and queries. [Read more](http://arxiv.org/abs/2405.13966v1).\n\n2. **Breaking ReAct Agents: Foot-in-the-Door Attack Will Get You In**: This study explores security vulnerabilities in ReAct agents, showing how they can be exploited using a foot-in-the-door attack. The research suggests implementing a reflection mechanism to mitigate such vulne